How do I restart your thinking on this project

For an ADC1115 what is the range of ADC counts 

In [4]:
single_ended_num_values = 32768
zero_offset = single_ended_num_values / 2
print(f"Zero offset for single-ended reading: {zero_offset}")
zero_offset_sampled = 12899
print(f"Sampled zero offset: {zero_offset_sampled}")
volts_per_count = 4.096 / single_ended_num_values
zero_offset_volts = zero_offset_sampled * volts_per_count
print(f"Zero offset in volts: {zero_offset_volts:0.3f} V")
bias_voltage = 1.61
print(f"Bias voltage: {bias_voltage:0.3f} V")
correction_voltage = bias_voltage - zero_offset_volts
print(f"Correction voltage: {correction_voltage:0.3f} V")
corrected_zero_offset_counts = zero_offset_sampled + (correction_voltage / volts_per_count)
print(f"Corrected zero offset: {corrected_zero_offset_counts:0.2f} counts")


Zero offset for single-ended reading: 16384.0
Sampled zero offset: 12899
Zero offset in volts: 1.612 V
Bias voltage: 1.610 V
Correction voltage: -0.002 V
Corrected zero offset: 12880.00 counts


In [8]:
maxV = bias_voltage *single_ended_num_values/ zero_offset_sampled 
print(f"Max voltage: {maxV:0.3f} V")  

1.61 / 12880
# 0.000125 V per count
lsb = 0.000125  # V per count for ADS1115 at gain of 1
print(f"LSB: {lsb:0.6f} V per count")

Max voltage: 4.090 V
LSB: 0.000125 V per count


This ADS115 in gain 1 mode can read 32768 values, up to 4.09 volts. But it is powered via the esp32 at 3.3V. What happens at voltages over 3.3

Ans: clipping at supply rail, max RMS from SCT013 15A is 1V, 1.4V peak or -1.4 peak

So all the ADC gives you is instantaneous reads 

A +/- 1.4 V sine wave reads 1 V RMS on the meter

A +/- 1.4 V sine wave reads 1 V RMS on the meter

The ADS1115 is taking instantaneous readings of samples anywhere between -1.4 V and +1.4V or 

In [9]:
V_plus = bias_voltage + 1.4
V_minus = bias_voltage - 1.4
ADC_plus = V_plus / lsb
ADC_minus = V_minus / lsb
print(f"ADC plus: {ADC_plus:0.1f} counts")
print(f"ADC minus: {ADC_minus:0.1f} counts")


ADC plus: 24080.0 counts
ADC minus: 1680.0 counts


ads.readADC_SingleEnded(0) reads for a +/- 1.4V signal riding on a 1.61 bias will be anywhere between 24080 and 1680.

How does taking 2000 samples ADC reads turn values in that range into an RMS voltage

Here's how to convert 2000 ADC samples into RMS voltage:

**Step-by-step process:**

1. **Subtract the DC bias (zero offset)** from each sample to get only the AC component:
   ```
   AC_sample = ADC_reading - zero_offset
   ```
   Example: 24080 - 12899 = 11181 counts (positive peak)

2. **Square each AC sample:**
   ```
   squared_sample = AC_sample²
   ```

3. **Average all 2000 squared samples:**
   ```
   mean_of_squares = (sum of all squared_samples) / 2000
   ```

4. **Take the square root to get RMS in ADC counts:**
   ```
   RMS_counts = √(mean_of_squares)
   ```

5. **Convert to voltage:**
   ```
   RMS_voltage = RMS_counts × LSB
   RMS_voltage = RMS_counts × 0.000125
   ```

**Example calculation:**
- For a ±1.4V peak sine wave:
- Peak in counts: ±11181 (relative to bias)
- RMS_counts = 11181 / √2 ≈ 7906 counts
- RMS_voltage = 7906 × 0.000125 ≈ **0.988V** (close to the expected 1.0V RMS)

**In code:**


In [ ]:
float sumSquares = 0.0;
for (int i = 0; i < 2000; i++) {
    int16_t reading = ads.readADC_SingleEnded(0);
    float acComponent = reading - zeroOffsetADC;
    sumSquares += acComponent * acComponent;
}
float rms_counts = sqrt(sumSquares / 2000.0);
float rms_voltage = rms_counts * 0.000125;

But aren't some of those samples occurring at other points on the 60hz sine wave. Wouldn't those readings, averaged ove 200 samples, substantially reduce the reading

Excellent question! The key is that we're **NOT** just averaging the samples - we're calculating the **Root Mean Square (RMS)**, which is specifically designed to handle this.

**Why it works:**

1. **We square each sample** before averaging:
   - Positive samples become positive squares
   - Negative samples also become positive squares
   - Zero-crossing samples contribute zero
   
2. **The squaring preserves the energy/power** at each instant, regardless of phase

3. **Example with a 60Hz sine wave:**

Let me show you with some sample points:



In [11]:
import numpy as np

# Simulate a 1V RMS sine wave (1.414V peak)
time_points = np.linspace(0, 1/60, 2000)  # 2000 samples over 1 cycle
sine_wave = 1.414 * np.sin(2 * np.pi * 60 * time_points)

# Method 1: Simple average (WRONG)
simple_avg = np.mean(sine_wave)
print(f"Simple average: {simple_avg:.4f}V")  # ≈ 0V (cancels out!)

# Method 2: RMS (CORRECT)
rms = np.sqrt(np.mean(sine_wave**2))
print(f"RMS: {rms:.4f}V")  # ≈ 1.0V (correct!)

print(sine_wave)

Simple average: 0.0000V
RMS: 0.9996V
[ 0.00000000e+00  4.44442691e-03  8.88880991e-03 ... -8.88880991e-03
 -4.44442691e-03 -1.60221440e-15]




**Result:**
- Simple average ≈ 0V (positive and negative cancel)
- RMS ≈ 1.0V (correct effective voltage)

**So sampling at different points is actually GOOD** - it gives you a representative sample across the entire waveform. The more samples and the more they're spread across multiple cycles, the more accurate your RMS calculation becomes.

For the ADS1115, if the LSB is .000125 at GAIN 1 what is it at other GAINS?

Here are the LSB (Least Significant Bit) values for all ADS1115 gain settings:

| Gain Setting | Voltage Range | LSB (V/count) | LSB (mV/count) |
|--------------|---------------|---------------|----------------|
| GAIN_TWOTHIRDS | ±6.144V | 0.0001875 | 0.1875 |
| GAIN_ONE | ±4.096V | 0.000125 | 0.125 |
| GAIN_TWO | ±2.048V | 0.0000625 | 0.0625 |
| GAIN_FOUR | ±1.024V | 0.00003125 | 0.03125 |
| GAIN_EIGHT | ±0.512V | 0.000015625 | 0.015625 |
| GAIN_SIXTEEN | ±0.256V | 0.0000078125 | 0.0078125 |

**Formula:**


In [ ]:
LSB = Full_Scale_Voltage / 32768



**For your application:**
- With a 1.61V bias and ±1.4V signal (max 3.01V), **GAIN_ONE** (±4.096V) is appropriate
- If you need more resolution for smaller currents, you could use **GAIN_TWO** (±2.048V), but watch for clipping at higher currents
- GAIN_TWOTHIRDS would give you headroom but lower resolution

**In code, you can use:**


In [ ]:
float lsbVolts;
switch(currentGain) {
    case GAIN_TWOTHIRDS: lsbVolts = 0.0001875; break;
    case GAIN_ONE:       lsbVolts = 0.000125;  break;
    case GAIN_TWO:       lsbVolts = 0.0000625; break;
    case GAIN_FOUR:      lsbVolts = 0.00003125; break;
    case GAIN_EIGHT:     lsbVolts = 0.000015625; break;
    case GAIN_SIXTEEN:   lsbVolts = 0.0000078125; break;
}

When you take a raw reading of ADC count via float rms_counts = sqrt(sumSquares / 2000.0);, at what GAIN is that reading being taken at?

ANS: This reads at the currently configured gain on the ADS1115, which is set by calling: ads.setGain(GAIN_ONE); 

For a heatpump where the current varies over an large range, is there a way to, inside loop(), take samples to determine if a recent change in current has stabilized at a value, then take samples at Gain 1 and if it is a low current, redo those samples at a more optimum gain setting?

I think I would like to construct the program so, inside loop(), it takes an reading of ADC counts

Yes! This is called **adaptive gain** or **auto-ranging**, and it's a great strategy for wide-range measurements. Here's how to implement it:

**Strategy:**

1. **Always start with GAIN_ONE** (widest range, ±4.096V) to avoid clipping
2. **Take a quick sample** (say 200 samples) to get RMS estimate
3. **If RMS is low**, switch to a higher gain for better resolution
4. **Retake full 2000 samples** at the optimized gain

**Implementation:**



In [ ]:
float readCurrentWithAutoGain() {
    const int QUICK_SAMPLES = 200;
    const int FULL_SAMPLES = 2000;
    
    // Step 1: Quick measurement at GAIN_ONE (safest, widest range)
    ads.setGain(GAIN_ONE);
    delay(10);
    
    float quickRMS = quickRMSMeasurement(QUICK_SAMPLES);
    float quickVoltage = quickRMS * 0.000125;  // LSB for GAIN_ONE
    
    // Step 2: Determine optimal gain based on signal level
    adsGain_t optimalGain = GAIN_ONE;
    float lsbVolts = 0.000125;
    
    if (quickVoltage < 0.15) {  // < 150mV RMS, use GAIN_SIXTEEN
        optimalGain = GAIN_SIXTEEN;
        lsbVolts = 0.0000078125;
    } else if (quickVoltage < 0.30) {  // < 300mV, use GAIN_EIGHT
        optimalGain = GAIN_EIGHT;
        lsbVolts = 0.000015625;
    } else if (quickVoltage < 0.60) {  // < 600mV, use GAIN_FOUR
        optimalGain = GAIN_FOUR;
        lsbVolts = 0.00003125;
    } else if (quickVoltage < 1.20) {  // < 1.2V, use GAIN_TWO
        optimalGain = GAIN_TWO;
        lsbVolts = 0.0000625;
    }
    // else stay at GAIN_ONE
    
    // Step 3: If gain changed, take full measurement at optimal gain
    if (optimalGain != GAIN_ONE) {
        ads.setGain(optimalGain);
        delay(10);
    }
    
    // Step 4: Full precision measurement
    float sumSquares = 0.0;
    for (int i = 0; i < FULL_SAMPLES; i++) {
        int16_t reading = ads.readADC_SingleEnded(0);
        float acComponent = reading - zeroOffsetADC;
        sumSquares += acComponent * acComponent;
    }
    
    float rms_counts = sqrt(sumSquares / FULL_SAMPLES);
    float rms_voltage = rms_counts * lsbVolts;
    
    // Convert to current using your calibration
    float current = rms_voltage * calibrationFactor;
    
    return current;
}

float quickRMSMeasurement(int numSamples) {
    float sumSquares = 0.0;
    for (int i = 0; i < numSamples; i++) {
        int16_t reading = ads.readADC_SingleEnded(0);
        float acComponent = reading - zeroOffsetADC;
        sumSquares += acComponent * acComponent;
    }
    return sqrt(sumSquares / numSamples);
}



**Thresholds explained:**
- Your bias is 1.61V, max signal is ±1.4V (3.01V total)
- At GAIN_TWO (±2.048V), you'd clip above ~0.44V RMS
- At GAIN_FOUR (±1.024V), you'd clip above ~0.22V RMS
- Choose thresholds with safety margin (20-30% below clipping)

**Benefits:**
- High resolution at low currents (fan only mode)
- No clipping at high currents (compressor running)
- Minimal overhead (quick 200-sample check first)

Would you like me to help integrate this into your main.cpp?

I will eventually get to that, I just want a readings at gain 1, for rms_counts and rms_volts.

But with this function for readCurrent()
```cpp
float readCurrent() {
  ads.setGain(GAIN_ONE);
  float lsbVolts = .000125; // LSB voltage for GAIN_ONE
  float sumSq = 0.0f;
  const int samples = 2000;
  float sumSquares = 0.0;
  for (int i = 0; i < samples; i++) {
      int16_t reading = ads.readADC_SingleEnded(0);
      float acComponent = reading - zeroOffsetADC;
      sumSquares += acComponent * acComponent;
  }
  float rms_counts = sqrt(sumSquares / samples);
  float rms_voltage = rms_counts * lsbVolts;
  Serial.printf("\t%.3f\t\t%d\t\t%s\n", 
    rms_voltage, (int)rms_counts, 
    currentGain == GAIN_LOW ? "4x" : "1x");
  delayMicroseconds(80); // ESP32 can sample faster
  return rms_voltage;
}
```
for a actual meter rms of .447 it gives
    rms_volts       rms_counts
     0.278           2223            1x

Your readings are showing **0.278V instead of the expected 0.447V**, which is off by a factor of about **1.6x** (0.447 / 0.278 = 1.607).

Several possible causes:

**1. Sampling Rate Issue**
The ADS1115 needs time between conversions. Add a small delay in your sampling loop:



In [ ]:
for (int i = 0; i < samples; i++) {
    int16_t reading = ads.readADC_SingleEnded(0);
    float acComponent = reading - zeroOffsetADC;
    sumSquares += acComponent * acComponent;
    delayMicroseconds(100);  // Add this! Give ADC time to convert
}



**2. Check Zero Offset**
Your `zeroOffsetADC` might have drifted. Add debug output in `readCurrent()`:



In [ ]:
Serial.printf("ZeroOffset: %.2f, ", zeroOffsetADC);



If it's wrong, recalibrate.

**3. Circuit Attenuation**
The 1.6x factor is suspicious. Check if there's an additional voltage divider or resistor network reducing your signal between the CT and the ADC input.

**4. Verify Peak Readings**
Add this temporary debug code to see actual ADC values:



In [ ]:
int16_t minReading = 32767, maxReading = -32768;
for (int i = 0; i < samples; i++) {
    int16_t reading = ads.readADC_SingleEnded(0);
    if (reading < minReading) minReading = reading;
    if (reading > maxReading) maxReading = reading;
    float acComponent = reading - zeroOffsetADC;
    sumSquares += acComponent * acComponent;
}
Serial.printf("Min: %d, Max: %d, Peak-to-Peak: %d counts\n", 
              minReading, maxReading, maxReading - minReading);



This will show you the actual voltage swing. For 0.447V RMS, you should see peak-to-peak ≈ 5050 counts (0.447 × √2 × 2 / 0.000125).

What values do you get?

Min: 9836, Max: 16145, Peak-to-Peak: 6309 counts
        0.276           2209            12904.41
Min: 9825, Max: 16161, Peak-to-Peak: 6336 counts
        0.272           2179            12904.41
Min: 9820, Max: 16161, Peak-to-Peak: 6341 counts
        0.200           1602            12904.41

In [14]:
minu = 12904.41 - 9825
plu = 16161.59 - 12904.41
print(f"Minu: {minu}, Plu: {plu}")
pp= minu + plu  
print(f"Peak-to-Peak: {pp}")


Minu: 3079.41, Plu: 3257.1800000000003
Peak-to-Peak: 6336.59


In [15]:
16161 * 1.61 / 12904.41
# 2.013 V max

2.0163037287252963

In [16]:
2.02-1.61

0.4099999999999999

As you can see from the debugging output I copied to ai.claude.ipynb, the peak to peak, while not quite the .447 ground truth, is  not .272. The ground truth measurement from the CT is taken at the input to the ADS115

If you're measuring **0.447V RMS directly at the ADS1115 input**, then the issue is clear:

**The ADC is under-reading by exactly 1.6x** (0.447 / 0.280 = 1.596)

This means:
- Expected peak for 0.447V RMS = 0.447 × √2 = **0.632V**
- Expected peak counts = 0.632 / 0.000125 = **5056 counts**
- You're measuring only **~3168 counts** (16161 - 12904)
- Ratio: 5056 / 3168 = **1.596x**

**Most likely cause: Gain setting not taking effect**

Try adding a longer delay after setting gain:



In [ ]:
ads.setGain(GAIN_ONE);
delay(100);  // Increase from 10ms to 100ms
ads.readADC_SingleEnded(0);  // Dummy read to flush
delay(10);



**Or verify the gain is actually set:**

Add this debug right before sampling:


In [ ]:
Serial.printf("Current gain setting check...\n");



**Alternative: The ADS1115 might be reverting to default GAIN_TWOTHIRDS**

If the ratio is consistently 1.5x-1.6x, the ADC may be using GAIN_TWOTHIRDS (LSB = 0.0001875) instead of GAIN_ONE (LSB = 0.000125).

Try using `0.0001875` as your LSB and see if the readings match!

Let's start again. main.cpp can calibrateZeroOffset() correctly. The rest doesn't work. What I would like to do is rewrite readCurrent() to take 2000 samples of the AC mV signal riding on top of the 1.61V DC bias. It is using an ADC1115 which 

# main.cpp stuck on 2/3

In [ ]:
#include <Arduino.h>
#include <Wire.h>
#include <Adafruit_ADS1X15.h>
// 


// ==================== HARDWARE CONFIGURATION ====================
// ESP32 I2C pins (default for DOIT DevKit v1)
#define I2C_SDA 21
#define I2C_SCL 22

// ADS1115 address (default 0x48, change if ADDR pin is connected)
#define ADS1115_ADDRESS 0x48
 // No argument
// CT Sensor Configuration
#define CT_RATIO 15.0f          // 15A per 1V (SCT-013-015)
#define LSB_VOLTAGE 0.125f          // 0.125 mV per count (GAIN_ONE)
#define BIAS_ADC_OFFSET 12893.0f    // Your zero offset
#define BIAS_VOLTAGE 1.612f         // Your measured bias voltage
#define VREF_LM4040 2.5f       // LM4040 reference voltage
#define R1_VALUE 10000.0f       // LM4040 series resistor (1.2kΩ)
#define R2_VALUE 87000.0f      // Bias divider resistor (10kΩ)
#define R3_VALUE 153000.0f      // Bias divider resistor (20kΩ)

// Measurement Configuration
#define SAMPLES_PER_READING 2000  // For RMS calculation
#define GAIN_SWITCH_THRESHOLD 2.0f // Amps: switch gain below this
#define CT_TURNS 1            // Number of primary turns (set to 2-3 for <2A)

// Fill this with measured values using a reference meter
struct CalPoint {
  float adcCounts;
  float actualCurrent;
};

// Example: Add 5-10 calibration points across your range
// Format: {ADC counts, actual current in amps}
const CalPoint calibrationTable[] = {
    {12893.0f, 0.00f},
    {13085.0f, 0.26f},
    {13181.0f, 0.48f},
    {13325.0f, 0.75f},
    {13469.0f, 1.04f},
    {13861.0f, 1.75f},
    {14213.0f, 2.39f},
    {15037.0f, 3.90f},
    {15341.0f, 4.45f},
    {16477.0f, 6.65f},
    {16717.0f, 7.12f},
    {17325.0f, 8.25f},
    {18821.0f, 10.90f},
    {19461.0f, 11.93f},
    {20397.0f, 13.80f},
};
const uint8_t numCalPoints = sizeof(calibrationTable) / sizeof(calibrationTable[0]);

// ==================== GLOBAL OBJECTS ====================
Adafruit_ADS1115 ads;
bool adsInitialized = false;

// Calibration values (run calibrateZeroOffset() on startup)
float zeroOffsetADC = 0.0f;
float temperature = 25.0f;  // For temp compensation if sensor added

// Gain state management
enum GainState { GAIN_LOW, GAIN_HIGH };
GainState currentGain = GAIN_HIGH;

// put function declarations here:
int myFunction(int, int);
void calibrateZeroOffset();
// float readCurrentWithAutoGain();
float readCurrent();
void switchGain(GainState newGain);
float getLsbVoltage();
float getEstimatedCurrent();  
float applyCalibration(float rawCurrent);


void setup() {
  Serial.begin(115200);
  Serial.println("\n\n=== Heat Pump Current Monitor v1.0 ===");
  pinMode(LED_BUILTIN, OUTPUT);
  // ads.begin(0x48); 
  int result = myFunction(2, 3);
    // Initialize I2C
  Wire.begin(I2C_SDA, I2C_SCL);
  Wire.setClock(400000); // 400kHz fast mode
  // Initialize ADS1115
  if (!ads.begin(ADS1115_ADDRESS, &Wire)) {
    Serial.println("ERROR: Failed to initialize ADS1115!");
    Serial.println("Check I2C wiring and ADDR pin connection");
    while (1) {
      delay(1000);
      ESP.restart();
    }
  }
  adsInitialized = true;
  ads.setGain(GAIN_ONE); // Start with ±1.024V range (16x for low current)
  currentGain = GAIN_HIGH;
  
  Serial.println("ADS1115 initialized successfully");
  
  // Calibrate zero offset (run with NO CURRENT through CT)
  Serial.println("Calibrating zero offset... ensure CT is not measuring any current");
  delay(3000);
  calibrateZeroOffset();
  
  Serial.println("\nSetup complete. Starting measurements...");
  Serial.println("Voltage[V]\tADC_Counts");
  Serial.println("-----------------------------------------------");
}

void loop() {
  digitalWrite(LED_BUILTIN, HIGH);
  delay(700);
  digitalWrite(LED_BUILTIN, LOW);
  delay(500);
    if (!adsInitialized) return;
  float current = readCurrent();
}

// put function definitions here:
int myFunction(int x, int y) {
  return x + y;
}

// ==================== CALIBRATION ====================
void calibrateZeroOffset() {
  const int numSamples = 500;
  float sum = 0.0f;
  
  // Ensure CT is NOT measuring current during calibration
  ads.setGain(GAIN_ONE); // Use most sensitive range for offset
  delay(50);
  
  for (int i = 0; i < numSamples; i++) {
    sum += ads.readADC_SingleEnded(0);
    delay(10);
  }
  
  zeroOffsetADC = sum / numSamples;
  
  Serial.printf("Zero offset calibrated: %.2f ADC counts\n", zeroOffsetADC);
  Serial.printf("This corresponds to: %.3f V bias\n", zeroOffsetADC * 0.000125f);
  
  // Restore gain setting
  ads.setGain(GAIN_ONE);
  currentGain = GAIN_HIGH;
} 

float readCurrent() {
  ads.setGain(GAIN_ONE);
  delay(100);  // Increase from 10ms to 100ms
  ads.readADC_SingleEnded(0);  // Dummy read to flush
  delay(10);
  float lsbVolts = .000125; // LSB voltage for GAIN_ONE
  float sumSq = 0.0f;
  const int samples = 2000;
  float sumSquares = 0.0;
  int16_t minReading = 32767, maxReading = -32768;
  // Serial.printf("Current gain setting check...\n");
  for (int i = 0; i < samples; i++) {
      int16_t reading = ads.readADC_SingleEnded(0);
      if (reading < minReading) minReading = reading;
      if (reading > maxReading) maxReading = reading;
      float acComponent = reading - zeroOffsetADC;
      sumSquares += acComponent * acComponent;
  }
  float rms_counts = sqrt(sumSquares / samples);
  float rms_voltage = rms_counts * lsbVolts;
  Serial.printf("\t%.3f\t\t%d\t\t%.2f\n", 
    rms_voltage, (int)rms_counts, zeroOffsetADC);
  Serial.printf("Min: %d, Max: %d, Peak-to-Peak: %d counts\n", 
      minReading, maxReading, maxReading - minReading);
  delayMicroseconds(80); // ESP32 can sample faster
  return rms_voltage;
}

// float readCurrentWithAutoGain() {
//   float sumSq = 0.0f;
//   int validSamples = 0;
  
//   // Auto-select gain based on expected current
//   float expectedCurrent = getEstimatedCurrent(); // Quick preliminary reading
//   GainState targetGain = (expectedCurrent < GAIN_SWITCH_THRESHOLD) ? GAIN_LOW : GAIN_HIGH;
  
//   if (targetGain != currentGain) {
//     switchGain(targetGain);
//   }
  
//   // Take RMS samples
//   for (int i = 0; i < SAMPLES_PER_READING; i++) {
//     int16_t rawADC = ads.readADC_SingleEnded(0);
    
//     // Remove bias and convert to voltage
//     float voltage = (rawADC - zeroOffsetADC) * getLsbVoltage();
    
//     // Convert to current (A = V / (1V/5A))
//     float currentInstant = voltage * CT_RATIO;
    
//     // Square for RMS
//     sumSq += currentInstant * currentInstant;
//     validSamples++;
    
//     // Maintain ~10kHz sampling rate
//     delayMicroseconds(100);
//   }
  
//   return sqrt(sumSq / validSamples);
// }

// ==================== GAIN MANAGEMENT ====================
void switchGain(GainState newGain) {
  currentGain = newGain;
  
  if (newGain == GAIN_LOW) {
    ads.setGain(GAIN_FOUR); // ±0.256V, 7.8125 µV/LSB
    Serial.println("\nSwitching to HIGH GAIN (±0.256V) for low current");
  } else {
    ads.setGain(GAIN_ONE); // ±1.024V, 31.25 µV/LSB
    Serial.println("\nSwitching to NORMAL GAIN (±1.024V) for high current");
  }
  
  delay(50); // Allow PGA to settle
}

float getLsbVoltage() {
  return (currentGain == GAIN_LOW) ? 0.0078125e-3f : 0.03125e-3f;
}

float getEstimatedCurrent() {
  // Quick single-sample reading for gain selection
  int16_t raw = ads.readADC_SingleEnded(0);
  float voltage = (raw - zeroOffsetADC) * getLsbVoltage();
  return abs(voltage) * CT_RATIO;
}

float applyCalibration(float rawCurrent) {
  // Multi-point linear interpolation
  if (numCalPoints < 2) return rawCurrent;
  
  // Find the bracket
  for (int i = 1; i < numCalPoints; i++) {
    if (rawCurrent <= calibrationTable[i].actualCurrent) {
      float x1 = calibrationTable[i-1].adcCounts;
      float y1 = calibrationTable[i-1].actualCurrent;
      float x2 = calibrationTable[i].adcCounts;
      float y2 = calibrationTable[i].actualCurrent;
      
      // Linear interpolation
      if (x2 - x1 != 0) {
        return y1 + (rawCurrent - x1) * (y2 - y1) / (x2 - x1);
      } else {
        return y1;
      }
    }
  }
  
  // Clamp to highest point
  return calibrationTable[numCalPoints-1].actualCurrent;
}

If the problem is the gain setting reverting to 2/3, could you give me a short main.cpp that I could use to se if the esp32 is able to set the gain on the ADS1115

Here's a minimal test program to verify gain settings on the ADS1115:



In [ ]:
#include <Arduino.h>
#include <Wire.h>
#include <Adafruit_ADS1X15.h>

#define I2C_SDA 21
#define I2C_SCL 22

Adafruit_ADS1115 ads;

void setup() {
  Serial.begin(115200);
  delay(1000);
  Serial.println("\n=== ADS1115 Gain Test ===");
  
  Wire.begin(I2C_SDA, I2C_SCL);
  
  if (!ads.begin(0x48)) {
    Serial.println("Failed to initialize ADS1115!");
    while (1) delay(1000);
  }
  
  Serial.println("ADS1115 found!");
  Serial.println("\nApply ~1.6V to AIN0");
  Serial.println("Testing each gain setting...\n");
  delay(3000);
}

void loop() {
  // Test all gain settings
  adsGain_t gains[] = {
    GAIN_TWOTHIRDS,  // ±6.144V
    GAIN_ONE,        // ±4.096V
    GAIN_TWO,        // ±2.048V
    GAIN_FOUR,       // ±1.024V
    GAIN_EIGHT,      // ±0.512V
    GAIN_SIXTEEN     // ±0.256V
  };
  
  const char* gainNames[] = {
    "2/3 (±6.144V)",
    "1x  (±4.096V)",
    "2x  (±2.048V)",
    "4x  (±1.024V)",
    "8x  (±0.512V)",
    "16x (±0.256V)"
  };
  
  float lsbValues[] = {
    0.1875,      // mV per count
    0.125,
    0.0625,
    0.03125,
    0.015625,
    0.0078125
  };
  
  Serial.println("Gain\t\tADC Count\tVoltage\t\tLSB used");
  Serial.println("-----------------------------------------------------------");
  
  for (int i = 0; i < 6; i++) {
    ads.setGain(gains[i]);
    delay(100);  // Let gain settle
    
    // Dummy read to flush
    ads.readADC_SingleEnded(0);
    delay(50);
    
    // Take 10 samples and average
    float sum = 0;
    for (int j = 0; j < 10; j++) {
      sum += ads.readADC_SingleEnded(0);
      delay(10);
    }
    int16_t avgCount = sum / 10;
    float voltage = avgCount * lsbValues[i] / 1000.0;  // Convert mV to V
    
    Serial.printf("%s\t%d\t\t%.4fV\t%.6f\n", 
                  gainNames[i], avgCount, voltage, lsbValues[i]/1000.0);
  }
  
  Serial.println("\n** ANALYSIS **");
  Serial.println("If gain is working:");
  Serial.println("  - All voltages should be ~1.6V (same)");
  Serial.println("  - ADC counts should DECREASE as gain increases");
  Serial.println("\nIf stuck at GAIN_TWOTHIRDS:");
  Serial.println("  - ADC counts will stay ~12900 for all gains");
  Serial.println("  - Only first reading will be correct\n");
  
  delay(10000);  // Wait 10 seconds before repeating
}



**What to look for in the output:**

**If gain is working correctly:**


In [ ]:
Gain            ADC Count    Voltage      LSB used
-----------------------------------------------------------
2/3 (±6.144V)   8580        1.609V       0.000188
1x  (±4.096V)   12870       1.609V       0.000125
2x  (±2.048V)   25740       1.609V       0.000063
4x  (±1.024V)   CLIPPED     CLIPPED      0.000031



**If stuck at GAIN_TWOTHIRDS (broken):**


In [ ]:
Gain            ADC Count    Voltage      LSB used
-----------------------------------------------------------
2/3 (±6.144V)   8580        1.609V       0.000188
1x  (±4.096V)   8580        1.073V       0.000125  ← WRONG!
2x  (±2.048V)   8580        0.536V       0.000063  ← WRONG!
4x  (±1.024V)   8580        0.268V       0.000031  ← WRONG!



Upload this and report what you see!

Gain            ADC Count       Voltage         LSB used
-----------------------------------------------------------
2/3 (±6.144V)   8648            1.6215V 0.000188
1x  (±4.096V)   12965           1.6206V 0.000125
2x  (±2.048V)   25854           1.6159V 0.000063
4x  (±1.024V)   32767           1.0240V 0.000031
8x  (±0.512V)   32767           0.5120V 0.000016
16x (±0.256V)   32767           0.2560V 0.000008

Summarized conversation history**Excellent news! Your ADS1115 gain control is working perfectly!** 

Analysis of your results:

✅ **GAIN_TWOTHIRDS**: 8648 counts → 1.621V  
✅ **GAIN_ONE**: 12965 counts → 1.621V (counts increased 1.5x as expected)  
✅ **GAIN_TWO**: 25854 counts → 1.616V (counts doubled again)  
❌ **GAIN_FOUR/EIGHT/SIXTEEN**: Clipping at 32767 (signal exceeds range)

The ADC counts are changing correctly with each gain setting, and all three working gains calculate ~1.62V (your bias voltage). **The gain register is accepting writes properly.**

---

**So what's causing the 0.447V vs 0.278V discrepancy?**

Since gain control works, let's investigate:

**1. Verify measurement points are identical:**
- Where exactly are you measuring with the multimeter?
- Is it at the same physical point as the ADS1115 AIN0 input?

**2. Check AC coupling capacitor:**
The 1µF capacitor might still be charging/settling. Try:
- Wait 30-60 seconds after applying AC signal
- Measure again to see if multimeter reading has changed

**3. Multimeter loading effect:**
Your multimeter has input impedance (~10MΩ) that might affect the circuit. Try:
- Disconnect multimeter
- Run the ESP32 measurement
- See if ADC readings change

**4. Verify with scope if available:**
A scope would show the actual waveform and confirm the voltage.

**What are your peak readings now?** Run your original `readCurrent()` code again and check if the min/max/peak-to-peak counts have changed at all.

In [17]:
490/337


1.454005934718101

what should be .490 is reported by the code as .337
* It it at the same physical point as the ADS1115 AIN0 input
* even after >30sec with the same signal the values remain at 2/3
* even with the multimeter disconnected

## Debugging the 0.490V vs 0.337V discrepancy

Since the gain test proves the ADS1115 **can** change gains properly, but your actual measurement is still off by 1.45x (0.490/0.337), there must be something different happening during the actual `readCurrent()` sampling.

**Key observation:** The ratio 0.490/0.337 = 1.454 ≈ 1.5

This 1.5x factor is the exact ratio between GAIN_TWOTHIRDS and GAIN_ONE LSB values:
- 0.0001875 / 0.000125 = 1.5

**This suggests:** During the 2000-sample RMS measurement, the ADC is actually reading at GAIN_TWOTHIRDS, even though you set GAIN_ONE.

**Possible causes:**

1. **Timing issue in tight loop** - The gain test takes 10 samples with delays; readCurrent() takes 2000 samples in a tight loop
2. **Gain not fully settled** - 100ms delay might not be enough before starting rapid sampling
3. **Different code path** - The gain test uses different timing/sequence than readCurrent()

**Next diagnostic step:** Add debug output to readCurrent() to check what the ADC is actually reading.

In [18]:
# Calculate what the ADC should be reading vs what it actually reads
meter_reading = 0.490  # V RMS from multimeter
code_reading = 0.337   # V RMS from ESP32 code

ratio = meter_reading / code_reading
print(f"Discrepancy ratio: {ratio:.3f}")

# If gain is TWOTHIRDS but we calculate with ONE LSB
lsb_twothirds = 0.0001875
lsb_one = 0.000125
lsb_ratio = lsb_twothirds / lsb_one
print(f"LSB ratio (TWOTHIRDS/ONE): {lsb_ratio:.3f}")

# What RMS counts should we see?
expected_peak_voltage = meter_reading * 1.414  # RMS to peak
expected_rms_counts_at_gain_one = (meter_reading * 1.414) / (1.414 * lsb_one)
print(f"\nExpected RMS counts at GAIN_ONE: {expected_rms_counts_at_gain_one:.0f}")

# What if ADC is actually at TWOTHIRDS?
expected_rms_counts_at_gain_twothirds = (meter_reading * 1.414) / (1.414 * lsb_twothirds)
print(f"Expected RMS counts at GAIN_TWOTHIRDS: {expected_rms_counts_at_gain_twothirds:.0f}")

# If we read TWOTHIRDS counts but calculate with ONE LSB
voltage_if_wrong_lsb = expected_rms_counts_at_gain_twothirds * lsb_one
print(f"\nIf ADC at TWOTHIRDS but we use ONE LSB: {voltage_if_wrong_lsb:.3f}V")
print(f"Actual code reading: {code_reading:.3f}V")
print(f"Match: {abs(voltage_if_wrong_lsb - code_reading) < 0.01}")

Discrepancy ratio: 1.454
LSB ratio (TWOTHIRDS/ONE): 1.500

Expected RMS counts at GAIN_ONE: 3920
Expected RMS counts at GAIN_TWOTHIRDS: 2613

If ADC at TWOTHIRDS but we use ONE LSB: 0.327V
Actual code reading: 0.337V
Match: False


## Theory: Gain reverts during continuous sampling

**Hypothesis:** The gain might be reverting to GAIN_TWOTHIRDS during the rapid 2000-sample loop in `readCurrent()`, even though it works in the gain test (which uses slower sampling with delays).

**Test this:** Modify `readCurrent()` to check the actual voltage reading at both the start and end of the 2000 samples to see if there's a difference.

## Diagnostic: Test with TWOTHIRDS LSB value

**Quick test:** In your current `readCurrent()` code, temporarily change the LSB value from `0.000125` to `0.0001875` and see if the reading becomes correct.

If changing to `lsbVolts = 0.0001875;` makes the voltage read 0.490V instead of 0.337V, then we've confirmed the ADC is stuck at GAIN_TWOTHIRDS during rapid sampling, even though it works in the slower gain test.

**The mystery:** Why does `ads.setGain(GAIN_ONE)` work in the gain test but not in readCurrent()?

Possible explanations:
1. **Library bug** - setGain() doesn't persist through continuous fast reads
2. **I2C bus issue** - Gain register write fails silently during fast operation
3. **ADC chip defect** - Hardware issue with PGA control
4. **Timing** - Need even longer delay or different sequencing

I can order another ADS1115 and see. What else could I try


## Troubleshooting steps before ordering new hardware

Before getting a new ADS1115, try these diagnostics:

### 1. **Confirm the LSB test result**
Did you upload the modified code with `lsbVolts = 0.0001875`? 
- If YES and it now reads ~0.490V → Confirmed ADC stuck at GAIN_TWOTHIRDS
- If NO → Try this first to confirm the diagnosis

### 2. **Set gain once and never change it**
Set GAIN_TWOTHIRDS in setup() and use the correct LSB throughout:
```cpp
void setup() {
  // ... existing code ...
  ads.setGain(GAIN_TWOTHIRDS);  // Accept the default gain
  delay(100);
  // Don't change gain anywhere else
}

float readCurrent() {
  // Remove ads.setGain() call
  float lsbVolts = 0.0001875;  // Match GAIN_TWOTHIRDS
  // ... rest of code ...
}
```
**Result:** You'll lose some resolution but measurements should work correctly.

### 3. **Read back the config register to verify writes**
Add diagnostic code to confirm gain setting was actually written:
```cpp
// After ads.setGain(GAIN_ONE);
uint16_t config = ads.readRegister(ADS1X15_REG_POINTER_CONFIG);
uint16_t pga = (config >> 9) & 0x07;
Serial.printf("PGA bits: 0x%X (should be 0x02 for GAIN_ONE)\n", pga);
```

### 4. **Try slower I2C speed**
400kHz might be too fast for your setup:
```cpp
Wire.setClock(100000); // Try 100kHz instead of 400000
```

### 5. **Check power supply stability**
- Measure VDD on ADS1115 during sampling (should be stable 3.3V)
- Try adding 10µF capacitor close to ADS1115 VDD pin
- Ensure ESP32 can supply enough current

### 6. **Try differential mode**
Single-ended mode might have issues; try differential:
```cpp
int16_t reading = ads.readADC_Differential_0_1();
```

### 7. **Hardware checks**
- Verify I2C pullup resistors (4.7kΩ to 3.3V on SDA and SCL)
- Check all solder joints on ADS1115 board
- Try different I2C pins on ESP32
- Measure voltage on ADDR pin (should be GND for 0x48)

### 8. **Simplest workaround for now**
**Just use GAIN_TWOTHIRDS permanently:**
- Set it once in setup()
- Use LSB = 0.0001875 everywhere
- You still have ±6.144V range (plenty for your 1.61V bias + signal)
- Only downside: slightly less resolution (0.1875mV vs 0.125mV per count)

Which would you like to try first?

In the circuit, SCL form the esp32 connects directly to the SCL on the ADS1115. Same with the SDA. There is no added 4.7K resistor. Should there be?



### What to try:

**Option 1: Reduce I2C speed (easiest)**
```cpp
Wire.setClock(100000); // 100kHz instead of 400000
```


## Resolution: Ordering new ADS1115

Tried changing I2C speed to 100kHz - no change, still reading at GAIN_TWOTHIRDS during rapid sampling.

### What we learned:
1. ✅ ADS1115 I2C communication works
2. ✅ Gain switching works in slow test (10 samples with delays)
3. ❌ Gain stuck at TWOTHIRDS during rapid 2000-sample measurement
4. ❌ I2C speed change didn't fix it

### Workaround until new hardware arrives:
Use GAIN_TWOTHIRDS permanently with LSB = 0.0001875

```cpp
void setup() {
  Wire.setClock(100000);
  ads.setGain(GAIN_TWOTHIRDS);  // Just accept it
  // ...
}

float readCurrent() {
  // Don't call ads.setGain() here
  float lsbVolts = 0.0001875;  // LSB for GAIN_TWOTHIRDS
  // ... rest of RMS calculation
}
```

**Trade-off:** Slightly less resolution (0.1875mV vs 0.125mV per count), but measurements will be accurate.

**Next steps:**
- Order new ADS1115 module
- When it arrives, test with same code
- If new module works properly → defective chip confirmed
- If same issue → may be library/ESP32 compatibility issue

Calibrating zero offset... ensure CT is not measuring any current
Zero offset calibrated: 12882.69 ADC counts
This corresponds to: 2.416 V bias

Edit [](file:///c%3A/Users/mcken/OneDrive/Tech/aiot/v1/ct4/ai.claude.ipynb)

Good decision to take a break and get new hardware. We've thoroughly debugged this and confirmed the ADS1115 gain control has an issue during rapid sampling, even though it works fine in the slower test.

**In the meantime**, if you need the current monitor working, just use the workaround - set GAIN_TWOTHIRDS once and use LSB = 0.0001875. You'll still get accurate measurements, just slightly less resolution (which is perfectly fine for your heat pump monitoring application).

When the new ADS1115 arrives, we'll know immediately if it was a defective chip or something else. Good luck with the project!

Made changes.

Working sloppy code for 2/3

In [ ]:
#include <Arduino.h>
#include <Wire.h>
#include <Adafruit_ADS1X15.h>
// 


// ==================== HARDWARE CONFIGURATION ====================
// ESP32 I2C pins (default for DOIT DevKit v1)
#define I2C_SDA 21
#define I2C_SCL 22

// ADS1115 address (default 0x48, change if ADDR pin is connected)
#define ADS1115_ADDRESS 0x48
 // No argument
// CT Sensor Configuration
#define CT_RATIO 15.0f          // 15A per 1V (SCT-013-015)
#define LSB_VOLTAGE 0.125f          // 0.125 mV per count (GAIN_ONE)
#define BIAS_ADC_OFFSET 12893.0f    // Your zero offset
#define BIAS_VOLTAGE 1.612f         // Your measured bias voltage
#define VREF_LM4040 2.5f       // LM4040 reference voltage
#define R1_VALUE 10000.0f       // LM4040 series resistor (1.2kΩ)
#define R2_VALUE 87000.0f      // Bias divider resistor (10kΩ)
#define R3_VALUE 153000.0f      // Bias divider resistor (20kΩ)

// Measurement Configuration
#define SAMPLES_PER_READING 2000  // For RMS calculation
#define GAIN_SWITCH_THRESHOLD 2.0f // Amps: switch gain below this
#define CT_TURNS 1            // Number of primary turns (set to 2-3 for <2A)

// Fill this with measured values using a reference meter
struct CalPoint {
  float adcCounts;
  float actualCurrent;
};

// Example: Add 5-10 calibration points across your range
// Format: {ADC counts, actual current in amps}
const CalPoint calibrationTable[] = {
    {12893.0f, 0.00f},
    {13085.0f, 0.26f},
    {13181.0f, 0.48f},
    {13325.0f, 0.75f},
    {13469.0f, 1.04f},
    {13861.0f, 1.75f},
    {14213.0f, 2.39f},
    {15037.0f, 3.90f},
    {15341.0f, 4.45f},
    {16477.0f, 6.65f},
    {16717.0f, 7.12f},
    {17325.0f, 8.25f},
    {18821.0f, 10.90f},
    {19461.0f, 11.93f},
    {20397.0f, 13.80f},
};
const uint8_t numCalPoints = sizeof(calibrationTable) / sizeof(calibrationTable[0]);

// ==================== GLOBAL OBJECTS ====================
Adafruit_ADS1115 ads;
bool adsInitialized = false;

// Calibration values (run calibrateZeroOffset() on startup)
float zeroOffsetADC = 0.0f;
float temperature = 25.0f;  // For temp compensation if sensor added

// Gain state management
enum GainState { GAIN_LOW, GAIN_HIGH };
GainState currentGain = GAIN_HIGH;

// put function declarations here:
int myFunction(int, int);
void calibrateZeroOffset();
// float readCurrentWithAutoGain();
float readCurrent();
void switchGain(GainState newGain);
float getLsbVoltage();
float getEstimatedCurrent();  
float applyCalibration(float rawCurrent);


void setup() {
  Serial.begin(115200);
  Serial.println("\n\n=== Heat Pump Current Monitor v1.0 ===");
  pinMode(LED_BUILTIN, OUTPUT);
  // ads.begin(0x48); 
  int result = myFunction(2, 3);
    // Initialize I2C
  Wire.begin(I2C_SDA, I2C_SCL);
  Wire.setClock(400000); 
  // Initialize ADS1115
  if (!ads.begin(ADS1115_ADDRESS, &Wire)) {
    Serial.println("ERROR: Failed to initialize ADS1115!");
    Serial.println("Check I2C wiring and ADDR pin connection");
    while (1) {
      delay(1000);
      ESP.restart();
    }
  }
  adsInitialized = true;
  ads.setGain(GAIN_ONE); // Start with ±1.024V range (16x for low current)
  currentGain = GAIN_HIGH;
  
  Serial.println("ADS1115 initialized successfully");
  
  // Calibrate zero offset (run with NO CURRENT through CT)
  Serial.println("Calibrating zero offset... ensure CT is not measuring any current");
  delay(3000);
  calibrateZeroOffset();
  
  Serial.println("\nSetup complete. Starting measurements...");
  Serial.println("Voltage[V]\tADC_Counts");
  Serial.println("-----------------------------------------------");
}

void loop() {
  digitalWrite(LED_BUILTIN, HIGH);
  delay(700);
  digitalWrite(LED_BUILTIN, LOW);
  delay(500);
    if (!adsInitialized) return;
  float current = readCurrent();
}

// put function definitions here:
int myFunction(int x, int y) {
  return x + y;
}

// ==================== CALIBRATION ====================
void calibrateZeroOffset() {
  const int numSamples = 500;
  float sum = 0.0f;
  
  // Ensure CT is NOT measuring current during calibration
  ads.setGain(GAIN_ONE); // Use most sensitive range for offset
  delay(50);
  
  for (int i = 0; i < numSamples; i++) {
    sum += ads.readADC_SingleEnded(0);
    delay(10);
  }
  
  zeroOffsetADC = sum / numSamples;
  
  Serial.printf("Zero offset calibrated: %.2f ADC counts\n", zeroOffsetADC);
  Serial.printf("This corresponds to: %.3f V bias\n", zeroOffsetADC * 0.000125
    f);
  
  // Restore gain setting
  ads.setGain(GAIN_ONE);
  currentGain = GAIN_HIGH;
} 

float readCurrent() {
  ads.setGain(GAIN_ONE);
  delay(100);  // Increase from 10ms to 100ms
  ads.readADC_SingleEnded(0);  // Dummy read to flush
  delay(10);
  float lsbVolts = 0.0001875; // LSB voltage for GAIN_ONE
  float sumSq = 0.0f;
  const int samples = 2000;
  float sumSquares = 0.0;
  int16_t minReading = 32767, maxReading = -32768;
  // Serial.printf("Current gain setting check...\n");
  for (int i = 0; i < samples; i++) {
      int16_t reading = ads.readADC_SingleEnded(0);
      if (reading < minReading) minReading = reading;
      if (reading > maxReading) maxReading = reading;
      float acComponent = reading - zeroOffsetADC;
      sumSquares += acComponent * acComponent;
  }
  float rms_counts = sqrt(sumSquares / samples);
  float rms_voltage = rms_counts * lsbVolts;
  Serial.printf("\t%.3f\t\t%d\t\t%.2f\n", 
    rms_voltage, (int)rms_counts, zeroOffsetADC);
  Serial.printf("Min: %d, Max: %d, Peak-to-Peak: %d counts\n", 
      minReading, maxReading, maxReading - minReading);
  delayMicroseconds(80); // ESP32 can sample faster
  return rms_voltage;
}

// float readCurrentWithAutoGain() {
//   float sumSq = 0.0f;
//   int validSamples = 0;
  
//   // Auto-select gain based on expected current
//   float expectedCurrent = getEstimatedCurrent(); // Quick preliminary reading
//   GainState targetGain = (expectedCurrent < GAIN_SWITCH_THRESHOLD) ? GAIN_LOW : GAIN_HIGH;
  
//   if (targetGain != currentGain) {
//     switchGain(targetGain);
//   }
  
//   // Take RMS samples
//   for (int i = 0; i < SAMPLES_PER_READING; i++) {
//     int16_t rawADC = ads.readADC_SingleEnded(0);
    
//     // Remove bias and convert to voltage
//     float voltage = (rawADC - zeroOffsetADC) * getLsbVoltage();
    
//     // Convert to current (A = V / (1V/5A))
//     float currentInstant = voltage * CT_RATIO;
    
//     // Square for RMS
//     sumSq += currentInstant * currentInstant;
//     validSamples++;
    
//     // Maintain ~10kHz sampling rate
//     delayMicroseconds(100);
//   }
  
//   return sqrt(sumSq / validSamples);
// }

// ==================== GAIN MANAGEMENT ====================
void switchGain(GainState newGain) {
  currentGain = newGain;
  
  if (newGain == GAIN_LOW) {
    ads.setGain(GAIN_FOUR); // ±0.256V, 7.8125 µV/LSB
    Serial.println("\nSwitching to HIGH GAIN (±0.256V) for low current");
  } else {
    ads.setGain(GAIN_ONE); // ±1.024V, 31.25 µV/LSB
    Serial.println("\nSwitching to NORMAL GAIN (±1.024V) for high current");
  }
  
  delay(50); // Allow PGA to settle
}

float getLsbVoltage() {
  return (currentGain == GAIN_LOW) ? 0.0078125e-3f : 0.03125e-3f;
}

float getEstimatedCurrent() {
  // Quick single-sample reading for gain selection
  int16_t raw = ads.readADC_SingleEnded(0);
  float voltage = (raw - zeroOffsetADC) * getLsbVoltage();
  return abs(voltage) * CT_RATIO;
}

float applyCalibration(float rawCurrent) {
  // Multi-point linear interpolation
  if (numCalPoints < 2) return rawCurrent;
  
  // Find the bracket
  for (int i = 1; i < numCalPoints; i++) {
    if (rawCurrent <= calibrationTable[i].actualCurrent) {
      float x1 = calibrationTable[i-1].adcCounts;
      float y1 = calibrationTable[i-1].actualCurrent;
      float x2 = calibrationTable[i].adcCounts;
      float y2 = calibrationTable[i].actualCurrent;
      
      // Linear interpolation
      if (x2 - x1 != 0) {
        return y1 + (rawCurrent - x1) * (y2 - y1) / (x2 - x1);
      } else {
        return y1;
      }
    }
  }
  
  // Clamp to highest point
  return calibrationTable[numCalPoints-1].actualCurrent;
}